### 1. 기본설치

In [ ]:
!pip install pytorch-lightning

In [ ]:
!pip install transformers

In [ ]:
!pip install torchmetrics

In [ ]:
!pip install kss

In [ ]:
import math
import pandas as pd
import numpy as np

from tqdm.auto import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import torch.optim as optim

from torch.nn.init import xavier_uniform_

from torchmetrics.functional import accuracy, f1, auroc
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

import kss

In [ ]:
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import pytorch_lightning as pl


### 2. 기본 설정

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

RANDOM_SEED = 42

sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8

pl.seed_everything(RANDOM_SEED)

Global seed set to 42


42

In [ ]:
MAX_TOKEN_COUNT = 512
N_EPOCHS = 10
BATCH_SIZE = 4

### 3. Data 가져오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/AiHub/P_NewsSum
%cd /
!cp /content/drive/MyDrive/Colab\ Notebooks/AiHub/P_NewsSum/NewsSumDs_json.zip  /content
%cd /content
!unzip -qq "NewsSumDs_json.zip"

/content/drive/MyDrive/Colab Notebooks/AiHub/P_NewsSum
/
/content


### 4. Data 처리

In [ ]:
import json
with open('/content/train.json', 'r', encoding = 'utf-8-sig') as json_file_train:
    json_data_train = json.load(json_file_train)
result_train = json_data_train

with open('/content/test.json', 'r', encoding = 'utf-8-sig') as json_file_test:
    json_data_test = json.load(json_file_test)
result_test = json_data_test

import pandas as pd
train_data = pd.DataFrame(result_train)
train_data = train_data.dropna()
test_data = pd.DataFrame(result_test)
test_data = test_data.dropna()
test_df = test_data

In [ ]:
train_df, test_df = train_test_split(train_data, test_size=0.05)
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)
train_df, val_df = train_test_split(train_data, test_size=0.05)
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
train_df.shape, val_df.shape, test_df.shape

((27344, 3), (1440, 3), (1440, 3))

In [ ]:
train_downsize = 2000 # 27344 -> 2000에서 100으로 조절
test_downsize = train_downsize//10
val_downsize = train_downsize//10
train_df = train_df[:train_downsize]
test_df = test_df[:test_downsize]
val_df = val_df[:val_downsize ]
train_df.shape, test_df.shape, val_df.shape

((2000, 3), (200, 3), (200, 3))

In [ ]:
tm1 = len(train_df['id'])

k = []
for i in range(tm1):
  x = train_df['article_original'][i]
  tm2 = len(x)
  y = train_df['extractive'][i]
  tm3 = len(y)
  t = []
  for j in range(tm3):
    if tm2 >= y[j]:
      tm = x[int(y[j])]
      t.append(tm)
  k.append(t)

train_df['abstractive'] = k

### 5. 모델

In [ ]:
from transformers import AutoTokenizer, AutoModel
BERT_MODEL_NAME = 'jinmang2/kpfbert'
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME)

Downloading:   0%|          | 0.00/270k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/335 [00:00<?, ?B/s]

In [ ]:
class SummDataset(Dataset):

    def __init__(
        self, 
        data: pd.DataFrame, 
        tokenizer: BertTokenizer, 
        max_token_len: int = 512
    ):
        self.tokenizer = tokenizer
        self.data = data
        self.max_token_len = max_token_len
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index: int):
        data_row = self.data.iloc[index]

        tokenlist = []
        for sent in data_row.article_original:
            tokenlist.append(tokenizer(
                text = sent,
                add_special_tokens = True))
    
        src = []
        labels = []
        segs = []
        clss = []

        odd = 0
        for tkns in tokenlist:
            if odd > 1 : odd = 0
            clss = clss + [len(src)]
            src = src + tkns['input_ids']
            segs = segs + [odd] * len(tkns['input_ids'])
            if tokenlist.index(tkns) in data_row.extractive :
                labels = labels + [1]
            else:
                labels = labels + [0]
            odd += 1
        
            #truncation
            if len(src) == MAX_TOKEN_COUNT:
                break
            elif len(src) > MAX_TOKEN_COUNT:
                src = src[:self.max_token_len - 1] + [src[-1]]
                segs = segs[:self.max_token_len]
                break
    
        #padding
        if len(src) < MAX_TOKEN_COUNT:
            src = src + [0]*(self.max_token_len - len(src))
            segs = segs + [0]*(self.max_token_len - len(segs))
            
        if len(clss) < MAX_TOKEN_COUNT:
            clss = clss + [-1]*(self.max_token_len - len(clss))
        if len(labels) < MAX_TOKEN_COUNT:
            labels = labels + [0]*(self.max_token_len - len(labels))

        return dict(
            src = torch.tensor(src),
            segs = torch.tensor(segs),
            clss = torch.tensor(clss),
            labels= torch.FloatTensor(labels)
        )

In [ ]:
class SummDataModule(pl.LightningDataModule):

    def __init__(self, train_df, test_df, val_df, tokenizer, batch_size=1, max_token_len=512):
        super().__init__()
        self.batch_size = batch_size
        self.train_df = train_df
        self.test_df = test_df
        self.val_df = val_df
        self.tokenizer = tokenizer
        self.max_token_len = max_token_len

    def setup(self, stage=None):
        self.train_dataset = SummDataset(
            self.train_df,
            self.tokenizer,
            self.max_token_len
        )

        self.test_dataset = SummDataset(
            self.test_df,
            self.tokenizer,
            self.max_token_len
        )
    
        self.val_dataset = SummDataset(
            self.val_df,
            self.tokenizer,
            self.max_token_len
        )

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=0 # windows는 0으로 고정해야 에러 안난다. num_workers=2
        )

    def val_dataloader(self):
        return DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            num_workers=0 # windows는 0으로 고정해야 에러 안난다. num_workers=2
        )

    def test_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            num_workers=0 # windows는 0으로 고정해야 에러 안난다. num_workers=2
        )

In [ ]:
data_module = SummDataModule(
  train_df,
  test_df,  
  val_df,
  tokenizer,
  batch_size=BATCH_SIZE,
  max_token_len=MAX_TOKEN_COUNT
)

In [ ]:
class PositionalEncoding(nn.Module):

    def __init__(self, dropout, dim, max_len=5000):
        pe = torch.zeros(max_len, dim)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp((torch.arange(0, dim, 2, dtype=torch.float) *
                              -(math.log(10000.0) / dim)))
        pe[:, 0::2] = torch.sin(position.float() * div_term)
        pe[:, 1::2] = torch.cos(position.float() * div_term)
        pe = pe.unsqueeze(0)
        super(PositionalEncoding, self).__init__()
        self.register_buffer('pe', pe)
        self.dropout = nn.Dropout(p=dropout)
        self.dim = dim

    def forward(self, emb, step=None):
        emb = emb * math.sqrt(self.dim)
        if (step):
            emb = emb + self.pe[:, step][:, None, :]

        else:
            emb = emb + self.pe[:, :emb.size(1)]
        emb = self.dropout(emb)
        return emb

    def get_emb(self, emb):
        return self.pe[:, :emb.size(1)]

In [ ]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, heads, d_ff, dropout):
        super(TransformerEncoderLayer, self).__init__()

        self.self_attn = MultiHeadedAttention(
            heads, d_model, dropout=dropout)
        self.feed_forward = PositionwiseFeedForward(d_model, d_ff, dropout)
        self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)
        self.dropout = nn.Dropout(dropout)

    def forward(self, iter, query, inputs, mask):
        if (iter != 0):
            input_norm = self.layer_norm(inputs)
        else:
            input_norm = inputs

        mask = mask.unsqueeze(1)
        context = self.self_attn(input_norm, input_norm, input_norm,
                                 mask=mask)
        out = self.dropout(context) + inputs
        return self.feed_forward(out)

In [ ]:
class ExtTransformerEncoder(nn.Module):
    def __init__(self, hidden_size=768, d_ff=2048, heads=8, dropout=0.2, num_inter_layers=2):
        super(ExtTransformerEncoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_inter_layers = num_inter_layers
        self.pos_emb = PositionalEncoding(dropout, hidden_size)
        self.transformer_inter = nn.ModuleList(
            [TransformerEncoderLayer(hidden_size, heads, d_ff, dropout)
            for _ in range(num_inter_layers)])
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(hidden_size, eps=1e-6)
        self.wo = nn.Linear(hidden_size, 1, bias=True)
        self.sigmoid = nn.Sigmoid()

    def forward(self, top_vecs, mask):
        """ See :obj:`EncoderBase.forward()`"""

        batch_size, n_sents = top_vecs.size(0), top_vecs.size(1)
        pos_emb = self.pos_emb.pe[:, :n_sents]
        x = top_vecs * mask[:, :, None].float()
        x = x + pos_emb

        for i in range(self.num_inter_layers):
            x = self.transformer_inter[i](i, x, x, ~mask) 

        x = self.layer_norm(x)
        sent_scores = self.sigmoid(self.wo(x))
        sent_scores = sent_scores.squeeze(-1) * mask.float()

        return sent_scores

In [ ]:
class PositionwiseFeedForward(nn.Module):
    """ A two-layer Feed-Forward-Network with residual layer norm.

    Args:
        d_model (int): the size of input for the first-layer of the FFN.
        d_ff (int): the hidden layer size of the second-layer
            of the FNN.
        dropout (float): dropout probability in :math:`[0, 1)`.
    """

    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)
        
    def gelu(self, x):
        return 0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))


    def forward(self, x):
        inter = self.dropout_1(self.gelu(self.w_1(self.layer_norm(x))))
        output = self.dropout_2(self.w_2(inter))
        return output + x

In [ ]:
class MultiHeadedAttention(nn.Module):
    """
    Multi-Head Attention module from
    "Attention is All You Need"
    :cite:`DBLP:journals/corr/VaswaniSPUJGKP17`.

    Similar to standard `dot` attention but uses
    multiple attention distributions simulataneously
    to select relevant items.

    .. mermaid::

       graph BT
          A[key]
          B[value]
          C[query]
          O[output]
          subgraph Attn
            D[Attn 1]
            E[Attn 2]
            F[Attn N]
          end
          A --> D
          C --> D
          A --> E
          C --> E
          A --> F
          C --> F
          D --> O
          E --> O
          F --> O
          B --> O

    Also includes several additional tricks.

    Args:
       head_count (int): number of parallel heads
       model_dim (int): the dimension of keys/values/queries,
           must be divisible by head_count
       dropout (float): dropout parameter
    """

    def __init__(self, head_count, model_dim, dropout=0.1, use_final_linear=True):
        assert model_dim % head_count == 0
        self.dim_per_head = model_dim // head_count
        self.model_dim = model_dim

        super(MultiHeadedAttention, self).__init__()
        self.head_count = head_count

        self.linear_keys = nn.Linear(model_dim,
                                     head_count * self.dim_per_head)
        self.linear_values = nn.Linear(model_dim,
                                       head_count * self.dim_per_head)
        self.linear_query = nn.Linear(model_dim,
                                      head_count * self.dim_per_head)
        self.softmax = nn.Softmax(dim=-1)
        self.dropout = nn.Dropout(dropout)
        self.use_final_linear = use_final_linear
        if (self.use_final_linear):
            self.final_linear = nn.Linear(model_dim, model_dim)

    def forward(self, key, value, query, mask=None,
                layer_cache=None, type=None, predefined_graph_1=None):
        """
        Compute the context vector and the attention vectors.

        Args:
           key (`FloatTensor`): set of `key_len`
                key vectors `[batch, key_len, dim]`
           value (`FloatTensor`): set of `key_len`
                value vectors `[batch, key_len, dim]`
           query (`FloatTensor`): set of `query_len`
                 query vectors  `[batch, query_len, dim]`
           mask: binary mask indicating which keys have
                 non-zero attention `[batch, query_len, key_len]`
        Returns:
           (`FloatTensor`, `FloatTensor`) :

           * output context vectors `[batch, query_len, dim]`
           * one of the attention vectors `[batch, query_len, key_len]`
        """

        batch_size = key.size(0)
        dim_per_head = self.dim_per_head
        head_count = self.head_count
        key_len = key.size(1)
        query_len = query.size(1)

        def shape(x):
            """  projection """
            return x.view(batch_size, -1, head_count, dim_per_head) \
                .transpose(1, 2)

        def unshape(x):
            """  compute context """
            return x.transpose(1, 2).contiguous() \
                .view(batch_size, -1, head_count * dim_per_head)

        # 1) Project key, value, and query.
        if layer_cache is not None:
            if type == "self":
                query, key, value = self.linear_query(query), \
                                    self.linear_keys(query), \
                                    self.linear_values(query)

                key = shape(key)
                value = shape(value)

                if layer_cache is not None:
                    device = key.device
                    if layer_cache["self_keys"] is not None:
                        key = torch.cat(
                            (layer_cache["self_keys"].to(device), key),
                            dim=2)
                    if layer_cache["self_values"] is not None:
                        value = torch.cat(
                            (layer_cache["self_values"].to(device), value),
                            dim=2)
                    layer_cache["self_keys"] = key
                    layer_cache["self_values"] = value
            elif type == "context":
                query = self.linear_query(query)
                if layer_cache is not None:
                    if layer_cache["memory_keys"] is None:
                        key, value = self.linear_keys(key), \
                                     self.linear_values(value)
                        key = shape(key)
                        value = shape(value)
                    else:
                        key, value = layer_cache["memory_keys"], \
                                     layer_cache["memory_values"]
                    layer_cache["memory_keys"] = key
                    layer_cache["memory_values"] = value
                else:
                    key, value = self.linear_keys(key), \
                                 self.linear_values(value)
                    key = shape(key)
                    value = shape(value)
        else:
            key = self.linear_keys(key)
            value = self.linear_values(value)
            query = self.linear_query(query)
            key = shape(key)
            value = shape(value)

        query = shape(query)

        key_len = key.size(2)
        query_len = query.size(2)

        # 2) Calculate and scale scores.
        query = query / math.sqrt(dim_per_head)
        scores = torch.matmul(query, key.transpose(2, 3))

        if mask is not None:
            mask = mask.unsqueeze(1).expand_as(scores)
            scores = scores.masked_fill(mask, -1e18) # how can i fix it to use fp16...

        # 3) Apply attention dropout and compute context vectors.

        attn = self.softmax(scores)

        if (not predefined_graph_1 is None):
            attn_masked = attn[:, -1] * predefined_graph_1
            attn_masked = attn_masked / (torch.sum(attn_masked, 2).unsqueeze(2) + 1e-9)

            attn = torch.cat([attn[:, :-1], attn_masked.unsqueeze(1)], 1)

        drop_attn = self.dropout(attn)
        if (self.use_final_linear):
            context = unshape(torch.matmul(drop_attn, value))
            output = self.final_linear(context)
            return output
        else:
            context = torch.matmul(drop_attn, value)
            return context

In [ ]:
class Summarizer(pl.LightningModule):

    def __init__(self, n_training_steps=None, n_warmup_steps=None):
        super().__init__()
        self.max_pos = 512
        self.bert = BertModel.from_pretrained(BERT_MODEL_NAME) #, return_dict=True)
        self.ext_layer = ExtTransformerEncoder()
        self.n_training_steps = n_training_steps
        self.n_warmup_steps = n_warmup_steps
        self.loss = nn.BCELoss(reduction='none')
    
        for p in self.ext_layer.parameters():
            if p.dim() > 1:
                xavier_uniform_(p)

    def forward(self, src, segs, clss, labels=None): #, input_ids, attention_mask, labels=None):
        
        mask_src = ~(src == 0) #1 - (src == 0)
        mask_cls = ~(clss == -1) #1 - (clss == -1)

        top_vec = self.bert(src, token_type_ids=segs, attention_mask=mask_src)
        top_vec = top_vec.last_hidden_state
        
        sents_vec = top_vec[torch.arange(top_vec.size(0)).unsqueeze(1), clss]
        sents_vec = sents_vec * mask_cls[:, :, None].float()

        sent_scores = self.ext_layer(sents_vec, mask_cls).squeeze(-1)
        
        loss = 0
        if labels is not None:
            loss = self.loss(sent_scores, labels)
            
            loss = (loss * mask_cls.float()).sum() / len(labels)
        
        return loss, sent_scores
    
    def step(self, batch):

        src = batch['src']
        if len(batch['labels']) > 0 :
            labels = batch['labels']
        else:
            labels = None
        segs = batch['segs']
        clss = batch['clss']
        
        loss, sent_scores = self(src, segs, clss, labels)    
        
        return loss, sent_scores, labels

    def training_step(self, batch, batch_idx):

        loss, sent_scores, labels = self.step(batch)
        self.log("train_loss", loss, prog_bar=True, logger=True)
        
        return {"loss": loss, "predictions": sent_scores, "labels": labels}

    def validation_step(self, batch, batch_idx):
        
        loss, sent_scores, labels = self.step(batch)
        self.log("val_loss", loss, prog_bar=True, logger=True)
        
        return {"loss": loss, "predictions": sent_scores, "labels": labels}

    def test_step(self, batch, batch_idx):
        
        loss, sent_scores, labels = self.step(batch)
        self.log("test_loss", loss, prog_bar=True, logger=True)
        
        return {"loss": loss, "predictions": sent_scores, "labels": labels}

    def acc_loss(self, outputs):
        total_loss = 0
        hit_cnt = 0
        for outp in outputs:
            labels = outp['labels'].cpu()
            predictions, idxs = outp['predictions'].cpu().sort()
            loss = outp['loss'].cpu()
            for label, idx in zip(labels, idxs):
                for i in range(1,3):
                    if label[idx[-i-1]] == 1 : 
                        hit_cnt += 1

            total_loss += loss
            
        avg_loss = total_loss / len(outputs)
        acc = hit_cnt / (3*len(outputs)*len(labels))
        
        return acc, avg_loss
        
    def training_epoch_end(self, outputs):
        
        acc, avg_loss = self.acc_loss(outputs)
        
        print('acc:', acc, 'avg_loss:', avg_loss)
        
        self.log('avg_train_loss', avg_loss, prog_bar=True, logger=True)

    def validation_epoch_end(self, outputs):
        
        acc, avg_loss = self.acc_loss(outputs)
        
        print('val_acc:', acc, 'avg_val_loss:', avg_loss)
        
        self.log('avg_val_loss', avg_loss, prog_bar=True, logger=True)

    def test_epoch_end(self, outputs):
        
        acc, avg_loss = self.acc_loss(outputs)
        
        print('test_acc:', acc, 'avg_test_loss:', avg_loss)
        
        self.log('avg_test_loss', avg_loss, prog_bar=True, logger=True)

        return
        
    def configure_optimizers(self):
        
        optimizer = AdamW(self.parameters(), lr=2e-5)

        steps_per_epoch=len(train_df) // BATCH_SIZE
        total_training_steps = steps_per_epoch * N_EPOCHS
        
        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=steps_per_epoch,
            num_training_steps=total_training_steps
        )

        return dict(
            optimizer=optimizer,
            lr_scheduler=dict(
                scheduler=scheduler,
                interval='step'
            )
        )

In [ ]:
model = Summarizer()

Downloading:   0%|          | 0.00/622 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of BertModel were not initialized from the model checkpoint at jinmang2/kpfbert and are newly initialized: ['pooler.dense.weight', 'pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 6. trainer 제작

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",
    filename="best-checkpoint",
    save_top_k=1,
    verbose=True,
    monitor="avg_val_loss",
    mode="min"
)

In [ ]:
logger = TensorBoardLogger("lightning_logs", name="kpfBERT_Summary")

In [ ]:
early_stopping_callback = EarlyStopping(monitor='avg_val_loss', patience=3)

### 7. 훈련(train) fit

In [ ]:
trainer = pl.Trainer(
    logger=logger,
    checkpoint_callback=checkpoint_callback,
    callbacks=[early_stopping_callback],
    max_epochs=N_EPOCHS,
    gpus=1,
    progress_bar_refresh_rate=30
)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:152: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7fc266e2f0d0>)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7fc266e2f0d0>)`.
  f"Setting `Trainer(checkpoint_callback={checkpoint_callback})` is deprecated in v1.5 and will "
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:97: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=30)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Tr

In [ ]:
trainer.fit(model, data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,

  | Name      | Type                  | Params
----------------------------------------------------
0 | bert      | BertModel             | 114 M 
1 | ext_layer | ExtTransformerEncoder | 11.0 M
2 | loss      | BCELoss               | 0     
----------------------------------------------------
125 M     Trainable params
0         Non-trainable params
125 M     Total params
500.230   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

val_acc: 0.20833333333333334 avg_val_loss: tensor(14.6913)


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

val_acc: 0.385 avg_val_loss: tensor(4.8646)
acc: 0.44866666666666666 avg_loss: tensor(3.3414)


Validation: 0it [00:00, ?it/s]

val_acc: 0.37666666666666665 avg_val_loss: tensor(6.6205)
acc: 0.5296666666666666 avg_loss: tensor(2.0657)


Validation: 0it [00:00, ?it/s]

val_acc: 0.38333333333333336 avg_val_loss: tensor(7.8647)
acc: 0.5936666666666667 avg_loss: tensor(1.0780)


In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./lightning_logs

### 8. 데이터셋 압축 & 복사

In [ ]:
%cd /content/lightning_logs
!zip -r /content/lightning_logs.zip /content/lightning_logs

/content/lightning_logs
  adding: content/lightning_logs/ (stored 0%)
  adding: content/lightning_logs/kpfBERT_Summary/ (stored 0%)
  adding: content/lightning_logs/kpfBERT_Summary/version_0/ (stored 0%)
  adding: content/lightning_logs/kpfBERT_Summary/version_0/hparams.yaml (stored 0%)
  adding: content/lightning_logs/kpfBERT_Summary/version_0/checkpoints/ (stored 0%)
  adding: content/lightning_logs/kpfBERT_Summary/version_0/checkpoints/epoch=5-step=3000.ckpt (deflated 13%)
  adding: content/lightning_logs/kpfBERT_Summary/version_0/events.out.tfevents.1649482966.841cb3b6d64f.74.0 (deflated 66%)


In [ ]:
!cp /content/lightning_logs.zip /content/drive/MyDrive/Colab\ Notebooks/AiHub/P_NewsSum


### 9. Prediction 준비

In [ ]:
trainer.checkpoint_callback.best_model_path = '/content/lightning_logs/kpfBERT_Summary/version_0/checkpoints/epoch=5-step=3000.ckpt'

In [ ]:
trained_model = Summarizer.load_from_checkpoint(
    trainer.checkpoint_callback.best_model_path
)
trained_model.eval()
trained_model.freeze()

Some weights of BertModel were not initialized from the model checkpoint at jinmang2/kpfbert and are newly initialized: ['pooler.dense.weight', 'pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def data_process(text):
    # 문장 분리 하고,
    sents = kss.split_sentences(text)
    
    #데이터 가공하고,
    tokenlist = []
    for sent in sents:
        tokenlist.append(tokenizer(
            text = sent,
            add_special_tokens = True)) #, # Add '[CLS]' and '[SEP]'

    src = [] # 토크나이징 된 전체 문단
    labels = []  # 요약문에 해당하면 1, 아니면 0으로 문장수 만큼 생성
    segs = []  #각 토큰에 대해 홀수번째 문장이면 0, 짝수번째 문장이면 1을 매핑
    clss = []  #[CLS]토큰의 포지션값을 지정

    odd = 0

    for tkns in tokenlist:

        if odd > 1 : odd = 0
        clss = clss + [len(src)]
        src = src + tkns['input_ids']
        segs = segs + [odd] * len(tkns['input_ids'])
        odd += 1

        #truncation
        if len(src) == MAX_TOKEN_COUNT:
            break
        elif len(src) > MAX_TOKEN_COUNT:
            src = src[:MAX_TOKEN_COUNT - 1] + [src[-1]]
            segs = segs[:MAX_TOKEN_COUNT]
            break

    #padding
    if len(src) < MAX_TOKEN_COUNT:
        src = src + [0]*(MAX_TOKEN_COUNT - len(src))
        segs = segs + [0]*(MAX_TOKEN_COUNT - len(segs))

    if len(clss) < MAX_TOKEN_COUNT:
        clss = clss + [-1]*(MAX_TOKEN_COUNT - len(clss))

    return dict(
        sents = sents, #정답 출력을 위해...
        src = torch.tensor(src),
        segs = torch.tensor(segs),
        clss = torch.tensor(clss),
    )

In [ ]:
def summarize_test(text):
    data = data_process(text.replace('\n',''))
    
    #trained_model에 넣어 결과값 반환
    _, rtn = trained_model(data['src'].unsqueeze(0), data['segs'].unsqueeze(0), data['clss'].unsqueeze(0))
    rtn = rtn.squeeze()
    
    # 예측 결과값을 받기 위한 프로세스
    rtn_sort, idx = rtn.sort(descending = True)
    
    rtn_sort = rtn_sort.tolist()
    idx = idx.tolist()

    end_idx = rtn_sort.index(0)

    rtn_sort = rtn_sort[:end_idx]
    idx = idx[:end_idx]
    
    if len(idx) > 3:
        rslt = idx[:3]
    else:
        rslt = idx
        
    summ = []
    print(' *** 입력한 문단의 요약문은 ...')
    for i, r in enumerate(rslt):
        summ.append(data['sents'][r])
        print('[', i+1, ']', summ[i])

    return summ

### 10. Prediction

In [ ]:
#테스트 문장 입력
test_context = '''세계 최대 전기차 업체 테슬라가 7일(현지시간) 미국 텍사스주 오스틴에 위치한 신규 생산공장 '기가 텍사스'의 가동을 알렸다. 기가 텍사스는 테슬라의 네 번째 생산 기지로 연간 50만대 생산이 목표다.

테슬라는 이날 기가 텍사스 준공 행사인 '사이버 로데오(Cyber Rodeo)'를 개최했다. 1만5000명 규모로 열린 이번 행사는 공장 탐방, 라이브 음악 행사 등 즐길 거리가 다양했다.

카우보이 모자와 선글라스를 쓴 채 테슬라 로드스터 2008을 타고 등장한 일론 머스크 테슬라 최고경영자(CEO)는 "기가 텍사스는 모델Y 생산에 힘입어 미국에서 가장 큰 생산능력을 갖춘 공장이 될 것"이라며 "내년에는 사이버트럭(픽업트럭) 생산도 시작한다"고 말했다. 이어 "세미(대형트럭), 로드스터(스포츠카)까지 내년에 모두 나온다"고 덧붙였다.

행사에서는 사이버트럭 양산형 모델의 실물도 공개됐다. 차 문 손잡이가 사라지는 등 디자인 변화가 있었는데, 손잡이가 없어도 주인을 인식해 자동으로 차 문을 열어준다는 설명이 뒤따랐다. 프란츠 폰 홀츠하우젠 테슬라 디자인 책임자는 사이버트럭에 대해 "엄청난 차가 될 것"이라며 "기다릴 만한 가치가 있다. 내가 장담한다"고 말했다. 사이버트럭을 사전예약한 사람은 100만명 이상으로 알려졌다.

텍사스 오스틴 공장은 테슬라가 11억달러(약 1조3000억원)를 투자해 만든 미국 내 두 번째 공장이다. 텍사스가 미국 내 신규 생산 기지로 낙점된 것은 기업들에 우호적인 지역이라서다. 텍사스는 규제가 까다롭지 않고 세금을 적게 거둬 기업하기 좋은 환경으로 평가된다. 타 지역 대비 물가도 저렴하다. 머스크 CEO가 지난해 본인 개인 거주지를 비롯해 테슬라 본사를 텍사스로 이전한 것도 이 때문이었다.

오스틴 공장에서는 전기 스포츠유틸리티차량(SUV) 모델Y와 사이버트럭 등이 생산된다. 이날 행사명이 '사이버 로데오'인 것도 사이버트럭 생산과 관련 있는 게 아니냐는 해석이 나왔다. 텍사스에서 생산되는 모델Y는 테슬라가 자체 개발한 '4680 배터리'가 적용된 신형 버전이다. 미국 환경부에 따르면 신형 모델Y(스탠다드 듀얼모터)의 주행거리는 279마일(약 450km)이다. 기존 모델Y 스탠다드 싱글모터보다 약 60km 늘었다.

독일 베를린 공장에 이어 오스틴 공장까지 신규 공장 2곳이 가동을 본격화하면서 테슬라의 올해 연간 생산량이 200만대에 달할 것이란 관측이 나온다. 지난해 생산 대수(93만448대)의 2배 이상 규모다. 미국 프리몬트 공장 60만대, 중국 상하이 공장 45만대, 독일 베를린 공장과 미국 텍사스 오스틴 공장이 50만대씩 생산 가능하다.

테슬라의 올 1분기 생산량은 30만5407대로 전년 동기 대비 69.4% 늘었다. 같은 기간 판매량은 31만48대로 지난해 1분기(18만4800대)보다 67.8% 늘어 1분기 기준 최다 기록을 세웠다. 제너럴모터스(GM), 도요타 등 글로벌 완성차 제조사가 반도체 공급 차질로 인해 고전하는 가운데 나홀로 상승세를 이어간 것. 자체 개발한 소프트웨어 기술력으로 차질을 최소화한 덕분이다.

최근 중국 상하이 봉쇄 장기화로 테슬라의 상하이 공장 중단 사태 또한 길어지고 있지만 업계에선 오스틴 공장과 베를린 공장이 가세하면서 감소분을 상쇄하면 목표 달성에 큰 지장은 없을 것으로 내다봤다.

지난달 14일 700달러선까지 밀렸던 테슬라 주가는 독일 베를린 공장과 오스틴 공장 개소 기대감에 힘입어 1000달러선을 회복했다. 이날 오스틴 공장 준공 소식에 테슬라 주가는 뉴욕증시에서 전거래일보다 1.10% 상승한 1057.26달러를 기록했다. 상승폭은 크지 않지만 경쟁업체인 리비안과 루시드가 각각 0.3%, 2.9% 하락 마감한 것을 고려하면 선방했다는 평가다. 앞서 독일 베를린 공장 준공식을 앞두고도 테슬라 주가는 7.9% 상승했다.
'''

In [ ]:
rtn = summarize_test(test_context)

[Korean Sentence Splitter]: Initializing Pynori...


 *** 입력한 문단의 요약문은 ...
[ 1 ] 기가 텍사스는 테슬라의 네 번째 생산 기지로 연간 50만대 생산이 목표다.
[ 2 ] 세계 최대 전기차 업체 테슬라가 7일(현지시간) 미국 텍사스주 오스틴에 위치한 신규 생산공장 '기가 텍사스'의 가동을 알렸다.
[ 3 ] 오스틴 공장에서는 전기 스포츠유틸리티차량(SUV) 모델Y와 사이버트럭 등이 생산된다.


In [ ]:
#테스트 문장 입력
test_context = '''미국의 전염병 권위자인 앤서니 파우치 국립알레르기·전염병연구소 소장이 올가을에 미국에서 코로나19의 재확산이 일어날 것이라고 경고했습니다.

파우치 소장은 블룸버그 TV에 출연해 관련 질문에 향후 몇 주간 확진자가 소폭 증가할 것으로 내다보며 이같이 예상했다고 CNN이 현지 시각 7일 보도했습니다.

파우치 소장은 "우리는 앞으로 몇 주간에 걸쳐 확진자가 소폭 증가할 것으로 예상해야 한다고 생각한다"며 "충분한 집단 면역이 형성돼 있어서 많은 입원으로 이어지지 않기를 바란다"고 말했습니다.

특히 미국이 영국 등 다른 나라의 추이를 뒤따라 밟아온 점, 마스크 착용 의무화와 다른 실내 방역 규제를 대거 해제한 점, 백신의 면역 효과가 시간의 경과에 따라 약화하는 점 등을 코로나19가 확산하기 좋은 여건으로 들었습니다.

파우치 소장은 다만 "소폭의 증가가 훨씬 더 많은 확진자가 나오는 확산으로 변할지는 예측하기 어렵다"면서도 가능성을 열어뒀습니다.

파우치 소장은 특히 올가을에는 지난 2차례의 가을과 비슷하게 코로나19의 확산이 있을 것으로 전망했습니다.
'''


In [ ]:
rtn = summarize_test(test_context)


 *** 입력한 문단의 요약문은 ...
[ 1 ] 미국의 전염병 권위자인 앤서니 파우치 국립알레르기·전염병연구소 소장이 올가을에 미국에서 코로나19의 재확산이 일어날 것이라고 경고했습니다.
[ 2 ] 파우치 소장은 블룸버그 TV에 출연해 관련 질문에 향후 몇 주간 확진자가 소폭 증가할 것으로 내다보며 이같이 예상했다고 CNN이 현지 시각 7일 보도했습니다.
[ 3 ] 특히 미국이 영국 등 다른 나라의 추이를 뒤따라 밟아온 점, 마스크 착용 의무화와 다른 실내 방역 규제를 대거 해제한 점, 백신의 면역 효과가 시간의 경과에 따라 약화하는 점 등을 코로나19가 확산하기 좋은 여건으로 들었습니다.
